In [1]:
from bs4 import BeautifulSoup
import urllib.parse
import urllib.request
import re
url = "https://www.allrecipes.com/recipe/15925/creamy-au-gratin-potatoes/"
#url = "https://www.allrecipes.com/recipe/277943/slow-cooker-loaded-potato-soup/?internalSource=previously%20viewed&referringContentType=Homepage&clickId=cardslot%2018"
#url = "https://www.allrecipes.com/recipe/273326/parmesan-crusted-shrimp-scampi-with-pasta/?internalSource=previously%20viewed&referringContentType=Homepage&clickId=cardslot%202"
#url = "https://www.allrecipes.com/recipe/277917/slow-cooker-cheesy-cauliflower-casserole/?internalSource=popular&referringContentType=Homepage&clickId=cardslot%205"
req = urllib.request.Request(url)
req.add_header('Cookie', 'euConsent=true')

html_content = urllib.request.urlopen(req).read()
soup = BeautifulSoup(html_content, 'html.parser')

In [2]:
ingredient_pattern = re.compile(r'itemprop="recipeIngredient">(?:(?!itemprop="recipeIngredient">|<\/span>)[\s\S])*<\/span>')

#ingredients = re.findall(ingredient_pattern, 'itemprop="recipeIngredient">1/4 cup chicken broth</span>')
ingredients = re.findall(ingredient_pattern, str(soup))

In [3]:
ingredients

['itemprop="recipeIngredient">4 russet potatoes, sliced into 1/4 inch slices</span>',
 'itemprop="recipeIngredient">1 onion, sliced into rings</span>',
 'itemprop="recipeIngredient">salt and pepper to taste</span>',
 'itemprop="recipeIngredient">3 tablespoons butter</span>',
 'itemprop="recipeIngredient">3 tablespoons all-purpose flour</span>',
 'itemprop="recipeIngredient">1/2 teaspoon salt</span>',
 'itemprop="recipeIngredient">2 cups milk</span>',
 'itemprop="recipeIngredient">1 1/2 cups shredded Cheddar cheese</span>']

In [4]:
len(ingredients)

8

In [5]:
update_ingredient = []

for i in range(len(ingredients)):
    print (ingredients[i])
    update_ingredient.append(ingredients[i][28:-7])

itemprop="recipeIngredient">4 russet potatoes, sliced into 1/4 inch slices</span>
itemprop="recipeIngredient">1 onion, sliced into rings</span>
itemprop="recipeIngredient">salt and pepper to taste</span>
itemprop="recipeIngredient">3 tablespoons butter</span>
itemprop="recipeIngredient">3 tablespoons all-purpose flour</span>
itemprop="recipeIngredient">1/2 teaspoon salt</span>
itemprop="recipeIngredient">2 cups milk</span>
itemprop="recipeIngredient">1 1/2 cups shredded Cheddar cheese</span>


In [6]:
update_ingredient

['4 russet potatoes, sliced into 1/4 inch slices',
 '1 onion, sliced into rings',
 'salt and pepper to taste',
 '3 tablespoons butter',
 '3 tablespoons all-purpose flour',
 '1/2 teaspoon salt',
 '2 cups milk',
 '1 1/2 cups shredded Cheddar cheese']

In [7]:
measurement = ['package', 'packages', 'carton', 'cartons', 'slice', 'slices', 'can', 'cans', 'teaspoon', 'tablespoon', 'cup', 'ounce', 'pint', 'quart', 'gallon', 'pound', 'dash', 'pinch', 'drop', 'peck', 'smidgen', 'saltspoon', 'scruple', 'coffeespoon', 'dessertspoon', 'wineglass', 'gill', 'teacup', 'pottle', 'dram', 'teaspoons', 'tablespoons', 'cups', 'ounces', 'pints', 'quarts', 'gallons', 'pounds', 'dashes', 'pinches', 'drops', 'pecks', 'smidgens', 'saltspoons', 'scruples', 'coffeespoons', 'dessertspoons', 'wineglasses', 'gills', 'teacups', 'pottles', 'drams']

In [8]:
len(measurement)

52

In [9]:
measure = []
ingred = []
quantity = []
for i in range(len(update_ingredient)):
    flag = True
    for item in measurement:
        if item in update_ingredient[i].lower().split():
            measure.append(item)
            ing_quant = update_ingredient[i].split(' '+item+' ')
            try: 
                ingred.append(ing_quant[1])
            except:
                flag = True
                break
            quantity.append(ing_quant[0])
            flag = False
            break
    if flag:
        measure.append("None")
        num_pattern = re.compile(r'\d+\/\d+|\d+')
        quan = re.findall(num_pattern, update_ingredient[i]) # only care about the first item, quant always shows at the beginning
        if quan:
            print (quan)
            quantity.append(quan[0])
            ingred.append(re.sub(quan[0]+' ', '', update_ingredient[i]))
        else:
            ingred.append(update_ingredient[i])
            
        
        

['4', '1/4']
['1']


In [10]:
measure

['slices',
 'None',
 'None',
 'None',
 'tablespoons',
 'tablespoons',
 'teaspoon',
 'cups',
 'cups']

In [11]:
ingred

['russet potatoes, sliced into 1/inch slices',
 'onion, sliced into rings',
 'salt and pepper to taste',
 'butter',
 'all-purpose flour',
 'salt',
 'milk',
 'shredded Cheddar cheese']

In [12]:
quantity

['4', '1', '3', '3', '1/2', '2', '1 1/2']

In [13]:
Descriptor = []
Preparation = []
refined_ingred = []
for item in ingred:
    ss = item.split(', ')
    if ss[0] != item:
        print (ss)
        refined_ingred.append(ss[0])
        Preparation.append(ss[1:])
    else:
        Preparation.append('None')
        refined_ingred.append(item)

['russet potatoes', 'sliced into 1/inch slices']
['onion', 'sliced into rings']


In [14]:
refined_ingred

['russet potatoes',
 'onion',
 'salt and pepper to taste',
 'butter',
 'all-purpose flour',
 'salt',
 'milk',
 'shredded Cheddar cheese']

In [15]:
Preparation

[['sliced into 1/inch slices'],
 ['sliced into rings'],
 'None',
 'None',
 'None',
 'None',
 'None',
 'None']

### nutrition

In [17]:
import requests
def splitUnit(string):
    for idx in range(len(string)):
        if not string[idx].isnumeric() and string[idx] != "." and string[idx] != "/":
            return string[:idx], string[idx:]
        
def scrape_nutrition(recipe_url):
        all_res = []
        # recipe_url.replace("?internalSource=hub%20recipe&referringId=86&referringContentType=Recipe%20Hub", "")
        # print (recipe_url)
        page_html = requests.get(recipe_url + "fullrecipenutrition/")
        page_graph = BeautifulSoup(page_html.content, "lxml")

        for row in page_graph.find_all("div", {"class":"nutrition-row"}):
            res = {}
            for span in row.find_all("span"):
                if span.get("class"):
                    key = span.get("class")[0]
                    val = span.text.split(":")[0]
                    if key == "nutrient-value":
                        res[key] = splitUnit(val)[0]
                        res["nutrient-unit"] = splitUnit(val)[1]
                    else:
                        res[key] = val
            all_res.append(res)
        
        return all_res

print(scrape_nutrition(url))

https://www.allrecipes.com/recipe/15925/creamy-au-gratin-potatoes/
[{'nutrient-name': 'Total Fat', 'nutrient-value': '25.4', 'nutrient-unit': 'g', 'daily-value': '39 %'}, {'nutrient-name': 'Saturated Fat', 'nutrient-value': '16.0', 'nutrient-unit': 'g'}, {'nutrient-name': 'Cholesterol', 'nutrient-value': '77', 'nutrient-unit': 'mg', 'daily-value': '26 %'}, {'nutrient-name': 'Sodium', 'nutrient-value': '683', 'nutrient-unit': 'mg', 'daily-value': '27 %'}, {'nutrient-name': 'Potassium', 'nutrient-value': '1184', 'nutrient-unit': 'mg', 'daily-value': '33 %'}, {'nutrient-name': 'Total Carbohydrates', 'nutrient-value': '49.3', 'nutrient-unit': 'g', 'daily-value': '16 %'}, {'nutrient-name': 'Dietary Fiber', 'nutrient-value': '4.4', 'nutrient-unit': 'g', 'daily-value': '17 %'}, {'nutrient-name': 'Protein', 'nutrient-value': '19.8', 'nutrient-unit': 'g', 'daily-value': '40 %'}, {'nutrient-name': 'Sugars', 'nutrient-value': '9', 'nutrient-unit': 'g'}, {'nutrient-name': 'Vitamin A', 'nutrient-va